# Librerias

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OrdinalEncoder

# Dataset

In [2]:
# Definicion de un path para acortar la definicion de la ruta
path = "\\Users\\Usuario\\Desktop\\hechos-delictivios-tdf\\"

In [3]:
pd.set_option('display.max_columns', None)
df = pd.read_csv(path + r"data\interim\TDF-hechos-delictivos.csv", delimiter =',')

# Limpieza de datos

In [4]:
# Las columnas son convertidas en minuscula
df.columns = df.columns.str.lower()

In [5]:
df.dtypes

provincia_id                  int64
provincia_nombre             object
departamento_id               int64
departamento_nombre          object
anio                          int64
                             ...   
total_exportacion_usd       float64
prod_pri_exportacion_usd    float64
origen_agro                 float64
origen_ind                  float64
combustible_y_energia       float64
Length: 65, dtype: object

In [6]:
df.isnull().sum()

provincia_id                  0
provincia_nombre              0
departamento_id               0
departamento_nombre           0
anio                          0
                           ... 
total_exportacion_usd       885
prod_pri_exportacion_usd    885
origen_agro                 885
origen_ind                  885
combustible_y_energia       885
Length: 65, dtype: int64

In [7]:
df.duplicated().sum()

0

### Eliminacion de registros
**Problematica:** "poblacion_por_departamento" es la unica columna con valores nulos, esto se debe a que en "departamento_nombre" existen cuatro valores: "Río Grande", "Ushuaia", "Tolhuin" y "Departamento sin determinar", sin embargo, la columna guarda unicamente registros de Río Grande y Ushuaia. 

**Solucion:** Eliminar los registros que corresponden a "detartamento sin determinar"

In [8]:
df.groupby("departamento_nombre")["cantidad_hechos"].sum()

departamento_nombre
Departamento sin determinar     102
Río Grande                     7707
Tolhuin                         155
Ushuaia                        3243
Name: cantidad_hechos, dtype: int64

In [9]:
# Eliminacion de los registros que corresponde a "Departamento sin determinar"
df = df.query('departamento_nombre not in ["Departamento sin determinar"]')

In [10]:
# Calculo del porcentaje de valores nulos
cant_nulos = df['poblacion_por_departamento'].isnull().sum()
cant_filas = len(df)
porcentaje = (cant_nulos / cant_filas) * 100

# Mostrar el resultado
print(f"Valores nulos: {cant_nulos}")
print(f"Total de filas: {cant_filas}")
print(f"Porcentaje de valores nulos: {porcentaje:.2f}%")

Valores nulos: 204
Total de filas: 1572
Porcentaje de valores nulos: 12.98%


**Problematica:** Aún existen valores nulos en la columna "poblacion_por_departamento" debido a la falta de datos sobre Tolhuin en los datasets extraidos del IPIEC, un pequeño porcentaje de las fuentes extraido del sitio guarda informacion unicamente de las ciudades de Río Grande y Ushuaia. Esto resulta un gran problema, pues la columna "nivel criminalistico" que califica el nivel de crimenes en bajo, alto y medio, se contruye utilizando la poblacion total del departamento.

**Solucion**: Se decide eliminar los registros correspondiente a Tolhuin que corresponde al 12.98% de la cantidad de hechos delictivos.

In [11]:
# Eliminacion de los registros que corresponde a "Tolhuin"
df = df.query('departamento_nombre not in ["Tolhuin"]')

In [12]:
df = df.rename(columns={"total": "impuesto_total", "mes_2": "mes_nombre"})

### Cambiar tipo de dato

In [13]:
columnas_con_comas = ['impuesto_total', 'impuesto ingresos brutos', 'convenio multilateral', 'impuesto de sellos', 'impuesto inmobiliario']

# Aplicar reemplazo de comas y conversión a entero en cada columna
df[columnas_con_comas] = df[columnas_con_comas].apply(lambda x: x.astype(str).str.replace(",", "").astype(float))

In [14]:
# Reemplazar valores desconocidos y convertir a float
df["población total"] = df["población total"].replace(["///", None, "NA"], pd.NA)  # Asegurar que todos los valores faltantes sean NaN
df["población total"] = pd.to_numeric(df["población total"], errors="coerce")  # Convertir la columna a números, ignorando errores

In [15]:
# Filtrar datos de 2020
datos_2020 = df[df["anio"] == 2020]
estimacion_t3 = (datos_2020.loc[datos_2020["trimestre"] == "2do trimestre", "población total"].values[0] +
                 datos_2020.loc[datos_2020["trimestre"] == "4to trimestre", "población total"].values[0]) / 2


In [16]:
df.loc[(df["anio"] == 2020) & (df["trimestre"] == "3er trimestre"), "población total"] = estimacion_t3

In [17]:
columnas_a_int = ["población total", "poblacion_por_departamento"]
df[columnas_a_int] = df[columnas_a_int].astype(int)

In [18]:
#En la columna "cba-linea de indigencia" y "cba-linea de pobreza" se detecto un valor cuyo formato es distinto al resto,
#con el fin de normalizar este valor con el resto de la misma columna, se hizo una correcion manual
df["cba-linea de indigencia"] = df["cba-linea de indigencia"].replace("11.503,89", 11503.89) 
df["cba-linea de pobreza"] = df["cba-linea de pobreza"].replace("29.910,11", 11503.89) 

# La columna "inversa del coeficiente de engel" tiene un registro de tipo de dato distinto al resto
# se utiliza la "," en lugar de "." para representar los decimales.
df["inversa del coeficiente de engel"] = df["inversa del coeficiente de engel"].str.replace(",", ".", regex=True)

columnas_a_float = ['cba-linea de indigencia','inversa del coeficiente de engel', 'cba-linea de pobreza',]
df[columnas_a_float] = df[columnas_a_float].astype(float)

### Redondear columnas

In [19]:
# Definir las columnas a modificar
columnas_a_redondear = ["cba-linea de indigencia","inversa del coeficiente de engel","cba-linea de pobreza", 'nivel general', 'alimentos y bebidas no alcohólicas',
       'bebidas alcohólicas y tabaco', 'prendas de vestir y calzado',
       'vivienda, agua, electricidad y otros combustibles',
       'equipamiento y mantenimiento del hogar', 'salud', 'transporte',
       'comunicación', 'recreación y cultura', 'educación',
       'restaurantes y hoteles', 'bienes y servicios varios']

# Redondear columnas a dos decimales
df[columnas_a_redondear] = df[columnas_a_redondear].round(2)

In [20]:
df.dtypes

provincia_id                  int64
provincia_nombre             object
departamento_id               int64
departamento_nombre          object
anio                          int64
                             ...   
total_exportacion_usd       float64
prod_pri_exportacion_usd    float64
origen_agro                 float64
origen_ind                  float64
combustible_y_energia       float64
Length: 65, dtype: object

In [21]:
df.head()

,provincia_id,provincia_nombre,departamento_id,departamento_nombre,anio,mes_nombre,codigo_delito_sat_prop,nombre_delito_sat_prop,cantidad_hechos,cantidad_hechos_inc_conocido,cantidad_hechos_inc_desconocido,cantidad_hechos_lugar_via_publ,cantidad_hechos_lugar_establec,cantidad_hechos_lugar_dom_part,cantidad_hechos_lugar_sd,cantidad_hechos_arma_de_fuego,cantidad_hechos_arma_otra,cantidad_hechos_arma_sin_arma,cantidad_hechos_arma_sd,cantidad_hechos_origen_denuncia,cantidad_hechos_origen_intervenc,cantidad_hechos_origen_orden_jud,cantidad_hechos_origen_otro,cant_hechos_agrav_por_lesiones,cant_hechos_agrav_sin_lesiones,cantidad_inculpados,cantidad_inculpados_sexo_masc,cantidad_inculpados_sexo_fem,cantidad_inculpados_sexo_sd,cantidad_inculpados_edad_0_15,cantidad_inculpados_edad_16_17,cantidad_inculpados_edad_mas_18,cantidad_inculpados_edad_sd,trimestre,población total,ocupados con ingresos,poblacion_por_departamento,mes,cba-linea de indigencia,inversa del coeficiente de engel,cba-linea de pobreza,nivel general,alimentos y bebidas no alcohólicas,bebidas alcohólicas y tabaco,prendas de vestir y calzado,"vivienda, agua, electricidad y otros combustibles",equipamiento y mantenimiento del hogar,salud,transporte,comunicación,recreación y cultura,educación,restaurantes y hoteles,bienes y servicios varios,impuesto_total,impuesto ingresos brutos,convenio multilateral,impuesto de sellos,impuesto inmobiliario,mes_y,total_exportacion_usd,prod_pri_exportacion_usd,origen_agro,origen_ind,combustible_y_energia
0,94,Tierra del Fuego,94007,Río Grande,2017,abril,6,Extorsiones,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2do trimestre,152940,62512,88768,4,1975.67,2.78,5492.36,108.65,105.48,109.66,105.76,119.35,103.66,110.68,107.55,115.26,108.96,123.68,107.69,108.84,596500020.0,63108045.0,235841071.0,20348071.0,25087.0,abril,2079780.64,0.0,0.0,2079780.64,0.0
1,94,Tierra del Fuego,94007,Río Grande,2017,abril,2,Hurto de automotores,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2do trimestre,152940,62512,88768,4,1975.67,2.78,5492.36,108.65,105.48,109.66,105.76,119.35,103.66,110.68,107.55,115.26,108.96,123.68,107.69,108.84,596500020.0,63108045.0,235841071.0,20348071.0,25087.0,abril,2079780.64,0.0,0.0,2079780.64,0.0
2,94,Tierra del Fuego,94007,Río Grande,2017,abril,1,Hurtos (excluir de automotores),70,10,60,12,14,44,0,0,0,70,0,46,24,0,0,0,0,10,7,3,0,0,0,10,0,2do trimestre,152940,62512,88768,4,1975.67,2.78,5492.36,108.65,105.48,109.66,105.76,119.35,103.66,110.68,107.55,115.26,108.96,123.68,107.69,108.84,596500020.0,63108045.0,235841071.0,20348071.0,25087.0,abril,2079780.64,0.0,0.0,2079780.64,0.0
3,94,Tierra del Fuego,94007,Río Grande,2017,abril,4,Robo de automotores,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2do trimestre,152940,62512,88768,4,1975.67,2.78,5492.36,108.65,105.48,109.66,105.76,119.35,103.66,110.68,107.55,115.26,108.96,123.68,107.69,108.84,596500020.0,63108045.0,235841071.0,20348071.0,25087.0,abril,2079780.64,0.0,0.0,2079780.64,0.0
4,94,Tierra del Fuego,94007,Río Grande,2017,abril,5,Robo de bancos,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2do trimestre,152940,62512,88768,4,1975.67,2.78,5492.36,108.65,105.48,109.66,105.76,119.35,103.66,110.68,107.55,115.26,108.96,123.68,107.69,108.84,596500020.0,63108045.0,235841071.0,20348071.0,25087.0,abril,2079780.64,0.0,0.0,2079780.64,0.0


In [22]:
# Ordenar dataset por orden de fecha
df = df.sort_values(by=["anio", "mes"])

In [23]:
# Se guarda el dataset
df.to_csv(path + r"data\processed\TDF-hechos-delictivos-limpio.csv", index=False)

# Ingenieria de caracteriscas

In [24]:
df = pd.read_csv(path + r"data\processed\TDF-hechos-delictivos-limpio.csv", delimiter =',')

## Nuevo dataset

### Consolidación de delitos en un nuevo dataset mensual por ciudad

Despues de la creacion de las anteriores columnas, se decidio hacer una simplificacion del dataset. En el dataset original se registran **9 tipos distintos de delitos**, entre ellos: hurtos, robos, secuestros, homicidios, robo de bancos, entre otros. Sin embargo, durante el análisis exploratorio se observó que:

- **La mayoría de los delitos (7 de 9 tipos)** presentan **baja frecuencia o escasa variabilidad**, con conteos generalmente entre 0 y 10 casos durante todo el período analizado (2017–2023).
- Solo **hurtos y robos** concentran una proporción significativa del total de hechos delictivos, representando el volumen principal del fenómeno.

**Decisión metodológica:**  
Dado que el objetivo es estudiar la evolución y características del nivel delictivo mensual en cada ciudad, se optó por **agrupar todos los tipos de delitos** en una única métrica: `cantidad_hechos`, que representa la **suma total de hechos delictivos ocurridos por mes y ciudad**, independientemente de su tipología.

Este enfoque permite:
- Obtener una métrica robusta y representativa del fenómeno delictivo a nivel agregado.
- Reducir el ruido estadístico provocado por delitos con muy baja frecuencia.
- Facilitar el análisis temporal y comparativo entre ciudades y períodos.


In [25]:
df.groupby("nombre_delito_sat_prop")["cantidad_hechos"].sum()

nombre_delito_sat_prop
Extorsiones                                               13
Hurto de automotores                                     260
Hurto de motocicletas                                      5
Hurtos (excluir de automotores)                         6642
Robo de automotores                                       15
Robo de bancos                                             0
Robo de motocicletas                                       5
Robos (excluir de automotores bancos y motocicletas)    4009
Secuestros                                                 1
Name: cantidad_hechos, dtype: int64

In [26]:
# 1. Seleccionar las columnas que son conteos y deben sumarse
columnas_a_sumar = [
    'cantidad_hechos',
    'cantidad_hechos_inc_conocido', 'cantidad_hechos_inc_desconocido',
    'cantidad_hechos_lugar_via_publ', 'cantidad_hechos_lugar_establec',
    'cantidad_hechos_lugar_dom_part', 'cantidad_hechos_lugar_sd',
    'cantidad_hechos_arma_de_fuego', 'cantidad_hechos_arma_otra',
    'cantidad_hechos_arma_sin_arma', 'cantidad_hechos_arma_sd',
    'cantidad_hechos_origen_denuncia', 'cantidad_hechos_origen_intervenc',
    'cantidad_hechos_origen_orden_jud', 'cantidad_hechos_origen_otro',
    'cant_hechos_agrav_por_lesiones', 'cant_hechos_agrav_sin_lesiones',
    'cantidad_inculpados', 'cantidad_inculpados_sexo_masc',
    'cantidad_inculpados_sexo_fem', 'cantidad_inculpados_sexo_sd',
    'cantidad_inculpados_edad_0_15', 'cantidad_inculpados_edad_16_17',
    'cantidad_inculpados_edad_mas_18', 'cantidad_inculpados_edad_sd'
]

# 2. Agrupar por ciudad, año y mes, sumando las columnas
df_simplificado = (
    df.groupby(['departamento_nombre', 'anio', 'mes'])[columnas_a_sumar]
    .sum()
    .reset_index()
)

# 3. Crear la columna de fecha
df_simplificado['fecha'] = pd.to_datetime(
    df_simplificado.rename(columns={'anio': 'year', 'mes': 'month'})[['year', 'month']].assign(day=1)
)

# 4. Ordenar por ciudad y fecha
df_simplificado = df_simplificado.sort_values(['departamento_nombre', 'fecha']).reset_index(drop=True)


### Integración de variables socioeconómicas

Además de los conteos de hechos delictivos e inculpados, el dataset original contenía variables socioeconómicas registradas por mes y por ciudad, como:

- Costo de la canasta básica (`cba`)
- Total de impuestos recaudados (`impuesto_total`)
- Cantidad de personas empleadas (`cant_personas_empleadas`)
- Población estimada del departamento (`poblacion_departamento`)

Estas variables fueron incorporadas al nuevo dataset mensual sin agregación, utilizando su valor promedio (o primero disponible) por mes y ciudad, ya que no representaban múltiples observaciones distintas dentro del mismo período.

In [27]:
columnas_socioeco = ['población total', 'trimestre', 'ocupados con ingresos',
       'poblacion_por_departamento', 'cba-linea de indigencia',
       'inversa del coeficiente de engel', 'cba-linea de pobreza',
       'nivel general', 'alimentos y bebidas no alcohólicas',
       'bebidas alcohólicas y tabaco', 'prendas de vestir y calzado',
       'vivienda, agua, electricidad y otros combustibles',
       'equipamiento y mantenimiento del hogar', 'salud', 'transporte',
       'comunicación', 'recreación y cultura', 'educación',
       'restaurantes y hoteles', 'bienes y servicios varios', 'impuesto_total',
       'impuesto ingresos brutos', 'convenio multilateral',
       'impuesto de sellos', 'impuesto inmobiliario','total_exportacion_usd', 'prod_pri_exportacion_usd', 'origen_agro',
       'origen_ind', 'combustible_y_energia']

df_socioeco = (
    df.groupby(['departamento_nombre', 'anio', 'mes'])[columnas_socioeco]
    .first()
    .reset_index()
)

In [28]:
# Union de dataset original y simplificado, a fin de conserver las variables socioeconómica mensual por ciudad
# Se hizo un merge para unir ambos desde el lado izquierdo segun departamento, anio y mes
df_simplificado = pd.merge(
    df_simplificado, df_socioeco,
    on=['departamento_nombre', 'anio', 'mes'],
    how='left'
)

In [29]:
df_simplificado.head(5)

,departamento_nombre,anio,mes,cantidad_hechos,cantidad_hechos_inc_conocido,cantidad_hechos_inc_desconocido,cantidad_hechos_lugar_via_publ,cantidad_hechos_lugar_establec,cantidad_hechos_lugar_dom_part,cantidad_hechos_lugar_sd,cantidad_hechos_arma_de_fuego,cantidad_hechos_arma_otra,cantidad_hechos_arma_sin_arma,cantidad_hechos_arma_sd,cantidad_hechos_origen_denuncia,cantidad_hechos_origen_intervenc,cantidad_hechos_origen_orden_jud,cantidad_hechos_origen_otro,cant_hechos_agrav_por_lesiones,cant_hechos_agrav_sin_lesiones,cantidad_inculpados,cantidad_inculpados_sexo_masc,cantidad_inculpados_sexo_fem,cantidad_inculpados_sexo_sd,cantidad_inculpados_edad_0_15,cantidad_inculpados_edad_16_17,cantidad_inculpados_edad_mas_18,cantidad_inculpados_edad_sd,fecha,población total,trimestre,ocupados con ingresos,poblacion_por_departamento,cba-linea de indigencia,inversa del coeficiente de engel,cba-linea de pobreza,nivel general,alimentos y bebidas no alcohólicas,bebidas alcohólicas y tabaco,prendas de vestir y calzado,"vivienda, agua, electricidad y otros combustibles",equipamiento y mantenimiento del hogar,salud,transporte,comunicación,recreación y cultura,educación,restaurantes y hoteles,bienes y servicios varios,impuesto_total,impuesto ingresos brutos,convenio multilateral,impuesto de sellos,impuesto inmobiliario,total_exportacion_usd,prod_pri_exportacion_usd,origen_agro,origen_ind,combustible_y_energia
0,Río Grande,2017,1,121,18,103,25,11,82,3,1,0,120,0,94,27,0,0,0,0,23,19,4,0,1,0,22,0,2017-01-01,151966,1er trimestre,60412,88768,1848.50,2.74,5064.89,102.56,100.95,101.19,100.91,110.32,101.07,104.73,102.48,101.66,100.96,106.99,102.27,102.48,519283888.0,71123083.0,158601571.0,21718644.0,22115.0,118334.49,0.0,0.0,118334.49,0.0
1,Río Grande,2017,2,116,20,96,15,14,84,3,0,0,116,0,87,29,0,0,0,0,21,18,3,0,0,3,18,0,2017-02-01,151966,1er trimestre,60412,88768,1882.42,2.76,5195.48,104.28,102.25,105.36,102.38,112.02,101.28,107.01,104.41,105.74,102.53,106.69,105.67,104.28,467864648.0,57416525.0,144090087.0,17667071.0,223034.0,6574.99,0.0,0.0,6574.99,0.0
2,Río Grande,2017,3,114,48,66,18,15,81,0,0,0,114,0,71,43,0,0,0,0,61,42,12,7,1,3,50,7,2017-03-01,151966,1er trimestre,60412,88768,1939.76,2.75,5334.34,106.52,103.98,106.56,104.79,115.37,102.31,108.73,106.31,107.84,105.03,123.32,105.46,106.40,482706552.0,69333643.0,145077379.0,17222376.0,149048.0,386758.81,0.0,0.0,386758.81,0.0
3,Río Grande,2017,4,154,26,128,28,32,94,0,1,1,152,0,105,49,0,0,0,0,26,19,7,0,0,1,25,0,2017-04-01,152940,2do trimestre,62512,88768,1975.67,2.78,5492.36,108.65,105.48,109.66,105.76,119.35,103.66,110.68,107.55,115.26,108.96,123.68,107.69,108.84,596500020.0,63108045.0,235841071.0,20348071.0,25087.0,2079780.64,0.0,0.0,2079780.64,0.0
4,Río Grande,2017,5,117,22,95,26,5,86,0,1,2,114,0,85,32,0,0,0,0,22,18,4,0,0,0,22,0,2017-05-01,152940,2do trimestre,62512,88768,1994.94,2.78,5545.93,110.49,107.60,111.71,106.96,123.20,105.85,112.56,108.07,115.39,112.57,123.89,108.00,109.86,584560675.0,67983284.0,194554548.0,23899008.0,9975.0,2548560.42,0.0,0.0,2548560.42,0.0


### Comparacion: df_simplificado vs df original

En la simplificacion del dataset original se perdieron o dejaron algunas columnas de lado tales como:
1. provincia_id: Tierra del Fuego tiene una sola provincia, por lo que no aporta variabilidad.
2. provincia_nombre: La provincia es constante.
3. departamento_id: Podría conservarse si necesitás un ID único. No es estrictamente necesario. 
4. mes_nombre: Ya está mes (numérico), más práctico para orden y análisis temporal.
5. codigo_delito_sat_prop: Eliminado a propósito al agrupar todos los delitos.
6. nombre_delito_sat_prop: Se decidió no distinguir tipos de delitos.

In [30]:
a , b = df.shape
print(f"""Dataframe original:
      Cantidad de filas: {a}
      Cantidad de columnas: {b}
      """)

Dataframe original:
      Cantidad de filas: 1368
      Cantidad de columnas: 65
      


In [31]:
a , b = df_simplificado.shape
print(f"""Dataframe simplificado:
      Cantidad de filas: {a}
      Cantidad de columnas: {b}
      """)

Dataframe simplificado:
      Cantidad de filas: 168
      Cantidad de columnas: 59
      


## Nuevas columnas

En esta etapa se crearan nuevas columnas a partir de columnas ya incorporadas en el dataset. Las siguientes columnas a crear son:
- cantidad_hechos_con_arma: esta columna simplifica las columnas de cantidad_hechos_arma_de_fuego y cantidad_hechos_arma_otra para registrar la existencia de la participion de un tipo de arma en el mes.
- inculpados_menores_edad: se simplifican las columnas cantidad_inculpados_edad_0_15 y cantidad_inculpados_edad_16_17 en una sola.
- cantidad_hechos_mes_pasado: Esta variable representa la cantidad de hechos delictivos registrados el mes anterior, para cada combinación de ciudad (departamento_nombre) y tipo de delito (nombre_delito_sat_prop). Los primeros registros de cada grupo (enero del 2017) no tendrán un valor anterior disponible, por lo tanto esta nueva columna tendrá valores nulos (NaN) en esos casos.
- desocupados: cantidad de personas que no tienen una ocupacion que genere ingresos monetarios, esta columna no distingue entre infantes y adultos.
- elecciones: indica sí hubo elecciones presidenciales y legislativa en el mes, junto con su asuncion o balotaje si lo hubiera.
- nivel_criminalistico: clasifica el nivel en tres categorias, bajo, medio y alto, esta clasificacion se realiza considerando el tamaño de la poblacion de cada departamento y la distribucion por cuartil.

In [32]:
# Asegurarse de que esté ordenado por ciudad y fecha
df_simplificado = df_simplificado.sort_values(['departamento_nombre', 'fecha'])

In [33]:
df_simplificado['cantidad_hechos_mes_pasado'] = (
    df_simplificado.groupby('departamento_nombre')['cantidad_hechos'].shift(1)
)
df_simplificado['cantidad_hechos_mes_pasado'] = df_simplificado['cantidad_hechos_mes_pasado'].astype('Int64')

In [34]:
df_simplificado['cantidad_hechos_con_arma'] = (
    df_simplificado['cantidad_hechos_arma_de_fuego'].fillna(0) +
    df_simplificado['cantidad_hechos_arma_otra'].fillna(0)
)

In [35]:
df_simplificado['inculpados_menores_edad'] = (
    df_simplificado['cantidad_inculpados_edad_0_15'].fillna(0) +
    df_simplificado['cantidad_inculpados_edad_16_17'].fillna(0)
)

In [36]:
df_simplificado['desocupados'] = df_simplificado['población total'] - df_simplificado['ocupados con ingresos']

In [38]:
df_simplificado.head()

,departamento_nombre,anio,mes,cantidad_hechos,cantidad_hechos_inc_conocido,cantidad_hechos_inc_desconocido,cantidad_hechos_lugar_via_publ,cantidad_hechos_lugar_establec,cantidad_hechos_lugar_dom_part,cantidad_hechos_lugar_sd,cantidad_hechos_arma_de_fuego,cantidad_hechos_arma_otra,cantidad_hechos_arma_sin_arma,cantidad_hechos_arma_sd,cantidad_hechos_origen_denuncia,cantidad_hechos_origen_intervenc,cantidad_hechos_origen_orden_jud,cantidad_hechos_origen_otro,cant_hechos_agrav_por_lesiones,cant_hechos_agrav_sin_lesiones,cantidad_inculpados,cantidad_inculpados_sexo_masc,cantidad_inculpados_sexo_fem,cantidad_inculpados_sexo_sd,cantidad_inculpados_edad_0_15,cantidad_inculpados_edad_16_17,cantidad_inculpados_edad_mas_18,cantidad_inculpados_edad_sd,fecha,población total,trimestre,ocupados con ingresos,poblacion_por_departamento,cba-linea de indigencia,inversa del coeficiente de engel,cba-linea de pobreza,nivel general,alimentos y bebidas no alcohólicas,bebidas alcohólicas y tabaco,prendas de vestir y calzado,"vivienda, agua, electricidad y otros combustibles",equipamiento y mantenimiento del hogar,salud,transporte,comunicación,recreación y cultura,educación,restaurantes y hoteles,bienes y servicios varios,impuesto_total,impuesto ingresos brutos,convenio multilateral,impuesto de sellos,impuesto inmobiliario,total_exportacion_usd,prod_pri_exportacion_usd,origen_agro,origen_ind,combustible_y_energia,cantidad_hechos_mes_pasado,cantidad_hechos_con_arma,inculpados_menores_edad,desocupados
0,Río Grande,2017,1,121,18,103,25,11,82,3,1,0,120,0,94,27,0,0,0,0,23,19,4,0,1,0,22,0,2017-01-01,151966,1er trimestre,60412,88768,1848.50,2.74,5064.89,102.56,100.95,101.19,100.91,110.32,101.07,104.73,102.48,101.66,100.96,106.99,102.27,102.48,519283888.0,71123083.0,158601571.0,21718644.0,22115.0,118334.49,0.0,0.0,118334.49,0.0,<NA>,1,1,91554
1,Río Grande,2017,2,116,20,96,15,14,84,3,0,0,116,0,87,29,0,0,0,0,21,18,3,0,0,3,18,0,2017-02-01,151966,1er trimestre,60412,88768,1882.42,2.76,5195.48,104.28,102.25,105.36,102.38,112.02,101.28,107.01,104.41,105.74,102.53,106.69,105.67,104.28,467864648.0,57416525.0,144090087.0,17667071.0,223034.0,6574.99,0.0,0.0,6574.99,0.0,121,0,3,91554
2,Río Grande,2017,3,114,48,66,18,15,81,0,0,0,114,0,71,43,0,0,0,0,61,42,12,7,1,3,50,7,2017-03-01,151966,1er trimestre,60412,88768,1939.76,2.75,5334.34,106.52,103.98,106.56,104.79,115.37,102.31,108.73,106.31,107.84,105.03,123.32,105.46,106.40,482706552.0,69333643.0,145077379.0,17222376.0,149048.0,386758.81,0.0,0.0,386758.81,0.0,116,0,4,91554
3,Río Grande,2017,4,154,26,128,28,32,94,0,1,1,152,0,105,49,0,0,0,0,26,19,7,0,0,1,25,0,2017-04-01,152940,2do trimestre,62512,88768,1975.67,2.78,5492.36,108.65,105.48,109.66,105.76,119.35,103.66,110.68,107.55,115.26,108.96,123.68,107.69,108.84,596500020.0,63108045.0,235841071.0,20348071.0,25087.0,2079780.64,0.0,0.0,2079780.64,0.0,114,2,1,90428
4,Río Grande,2017,5,117,22,95,26,5,86,0,1,2,114,0,85,32,0,0,0,0,22,18,4,0,0,0,22,0,2017-05-01,152940,2do trimestre,62512,88768,1994.94,2.78,5545.93,110.49,107.60,111.71,106.96,123.20,105.85,112.56,108.07,115.39,112.57,123.89,108.00,109.86,584560675.0,67983284.0,194554548.0,23899008.0,9975.0,2548560.42,0.0,0.0,2548560.42,0.0,154,3,0,90428


In [39]:
# Función para clasificar elecciones
def clasificar_elecciones(row):
    if row["anio"] in [2017, 2021] and row["mes"] == 10:
        return "legislativa"
    elif row["anio"] in [2019, 2023] and row["mes"] == 10:
        return "presidenciales"
    elif row["anio"] == 2023 and row["mes"] == 11:
        return "balotaje"
    elif row["mes"] == "12":
        return "asuncion presidencial"
    else:
        return "no"

df_simplificado["elecciones"] = df_simplificado.apply(clasificar_elecciones, axis=1)

### Creación de la variable `nivel_criminalístico`

Para facilitar la clasificación de los niveles delictivos entre distintos departamentos y períodos, se creó la variable `nivel_criminalístico`. Esta variable categoriza los niveles de criminalidad en tres grupos: **Bajo**, **Medio** y **Alto**.

#### Metodología:
1. Se calculó la **tasa de delitos por cada 100.000 habitantes**, utilizando la fórmula:

   \[
   \text{tasa\_delitos} = \left( \frac{\text{cantidad\_hechos}}{\text{poblacion\_por\_departamento}} \right) \times 100000
   \]

2. Esta tasa fue luego dividida en tres grupos de igual tamaño utilizando la función `pd.qcut()`, que segmenta la distribución en terciles.

3. Se asignaron etiquetas:
   - **Bajo**: tasa delictiva en el tercil inferior
   - **Medio**: tasa delictiva en el tercil medio
   - **Alto**: tasa delictiva en el tercil superior

#### Justificación:
Este enfoque permite una comparación proporcional entre regiones con diferente cantidad de habitantes, normalizando la cantidad de hechos delictivos y facilitando su análisis desde una perspectiva relativa en lugar de absoluta.


In [40]:
df_simplificado['tasa_delitos'] = df_simplificado['cantidad_hechos'] / df_simplificado['poblacion_por_departamento'] * 100000

In [41]:
df_simplificado['nivel_criminalistico'] = pd.qcut(
    df_simplificado['tasa_delitos'],
    q=3,
    labels=['Bajo', 'Medio', 'Alto']
)

In [42]:
df_simplificado.head()

,departamento_nombre,anio,mes,cantidad_hechos,cantidad_hechos_inc_conocido,cantidad_hechos_inc_desconocido,cantidad_hechos_lugar_via_publ,cantidad_hechos_lugar_establec,cantidad_hechos_lugar_dom_part,cantidad_hechos_lugar_sd,cantidad_hechos_arma_de_fuego,cantidad_hechos_arma_otra,cantidad_hechos_arma_sin_arma,cantidad_hechos_arma_sd,cantidad_hechos_origen_denuncia,cantidad_hechos_origen_intervenc,cantidad_hechos_origen_orden_jud,cantidad_hechos_origen_otro,cant_hechos_agrav_por_lesiones,cant_hechos_agrav_sin_lesiones,cantidad_inculpados,cantidad_inculpados_sexo_masc,cantidad_inculpados_sexo_fem,cantidad_inculpados_sexo_sd,cantidad_inculpados_edad_0_15,cantidad_inculpados_edad_16_17,cantidad_inculpados_edad_mas_18,cantidad_inculpados_edad_sd,fecha,población total,trimestre,ocupados con ingresos,poblacion_por_departamento,cba-linea de indigencia,inversa del coeficiente de engel,cba-linea de pobreza,nivel general,alimentos y bebidas no alcohólicas,bebidas alcohólicas y tabaco,prendas de vestir y calzado,"vivienda, agua, electricidad y otros combustibles",equipamiento y mantenimiento del hogar,salud,transporte,comunicación,recreación y cultura,educación,restaurantes y hoteles,bienes y servicios varios,impuesto_total,impuesto ingresos brutos,convenio multilateral,impuesto de sellos,impuesto inmobiliario,total_exportacion_usd,prod_pri_exportacion_usd,origen_agro,origen_ind,combustible_y_energia,cantidad_hechos_mes_pasado,cantidad_hechos_con_arma,inculpados_menores_edad,desocupados,elecciones,tasa_delitos,nivel_criminalistico
0,Río Grande,2017,1,121,18,103,25,11,82,3,1,0,120,0,94,27,0,0,0,0,23,19,4,0,1,0,22,0,2017-01-01,151966,1er trimestre,60412,88768,1848.50,2.74,5064.89,102.56,100.95,101.19,100.91,110.32,101.07,104.73,102.48,101.66,100.96,106.99,102.27,102.48,519283888.0,71123083.0,158601571.0,21718644.0,22115.0,118334.49,0.0,0.0,118334.49,0.0,<NA>,1,1,91554,no,136.310382,Alto
1,Río Grande,2017,2,116,20,96,15,14,84,3,0,0,116,0,87,29,0,0,0,0,21,18,3,0,0,3,18,0,2017-02-01,151966,1er trimestre,60412,88768,1882.42,2.76,5195.48,104.28,102.25,105.36,102.38,112.02,101.28,107.01,104.41,105.74,102.53,106.69,105.67,104.28,467864648.0,57416525.0,144090087.0,17667071.0,223034.0,6574.99,0.0,0.0,6574.99,0.0,121,0,3,91554,no,130.677722,Alto
2,Río Grande,2017,3,114,48,66,18,15,81,0,0,0,114,0,71,43,0,0,0,0,61,42,12,7,1,3,50,7,2017-03-01,151966,1er trimestre,60412,88768,1939.76,2.75,5334.34,106.52,103.98,106.56,104.79,115.37,102.31,108.73,106.31,107.84,105.03,123.32,105.46,106.40,482706552.0,69333643.0,145077379.0,17222376.0,149048.0,386758.81,0.0,0.0,386758.81,0.0,116,0,4,91554,no,128.424658,Alto
3,Río Grande,2017,4,154,26,128,28,32,94,0,1,1,152,0,105,49,0,0,0,0,26,19,7,0,0,1,25,0,2017-04-01,152940,2do trimestre,62512,88768,1975.67,2.78,5492.36,108.65,105.48,109.66,105.76,119.35,103.66,110.68,107.55,115.26,108.96,123.68,107.69,108.84,596500020.0,63108045.0,235841071.0,20348071.0,25087.0,2079780.64,0.0,0.0,2079780.64,0.0,114,2,1,90428,no,173.485941,Alto
4,Río Grande,2017,5,117,22,95,26,5,86,0,1,2,114,0,85,32,0,0,0,0,22,18,4,0,0,0,22,0,2017-05-01,152940,2do trimestre,62512,88768,1994.94,2.78,5545.93,110.49,107.60,111.71,106.96,123.20,105.85,112.56,108.07,115.39,112.57,123.89,108.00,109.86,584560675.0,67983284.0,194554548.0,23899008.0,9975.0,2548560.42,0.0,0.0,2548560.42,0.0,154,3,0,90428,no,131.804254,Alto


## Eliminacion de columnas base

Luego de crear nuevas variables simplificadas a partir de columnas originales —como `inculpados_menores_edad` (a partir de `cantidad_inculpados_edad_0_15` y `cantidad_inculpados_edad_16_17`) y `cantidad_hechos_con_arma` (a partir de `cantidad_hechos_arma_de_fuego` y `cantidad_hechos_arma_otra`)— se procede a eliminar las columnas que sirvieron como base. Estas columnas ya no son necesarias para el análisis, y su eliminación contribuye a reducir la dimensionalidad del dataset, facilitando la interpretación y el procesamiento posterior.


In [43]:
columnas_a_eliminar = [
    'cantidad_inculpados_edad_0_15',
    'cantidad_inculpados_edad_16_17',
    'cantidad_hechos_arma_de_fuego',
    'cantidad_hechos_arma_otra',
]

df_simplificado = df_simplificado.drop(columns=columnas_a_eliminar)


In [44]:
a , b = df_simplificado.shape
print(f"""Dataframe simplificado:
      Cantidad de filas: {a}
      Cantidad de columnas: {b}
      """)

Dataframe simplificado:
      Cantidad de filas: 168
      Cantidad de columnas: 62
      


In [45]:
# Se guarda el nuevo dataframe simplificado.
df_simplificado.to_csv(path + r"data\processed\TDF-hechos-delictivos-simplificado.csv", index=False)